In [1]:
# Parameters
Rs = None
sza = None
# papermill parameters

In [2]:
# Parameters
Rs = 0.2
sza = -90.0


In [3]:
import os
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import sys
import math
from netCDF4 import Dataset
import os, calendar, sys, fnmatch, datetime
from sur_spec_alb_interpolate import sur_spec_alb_interpolate

#python scrapes
from INPUT_RRTM_sw import *
from INPUT_CLD_RRTM_sw import *
from INPUT_AER_RRTM_sw import *
from read_output_rrtm import *

In [4]:
#******************* Specify the path of compiled RRTM codes ****************
model_dir = '../../../../RRTM/'
RT_model_sw = model_dir+'rrtm_sw/rrtm_sw_OS_X_v2.5'

#=======================================
#Specify parameters in the RRTM model
#=======================================

SOLVAR  = 0.0  #(16,29)the solar source function scale factor for each band.
#HBOUND = np.loadtxt(fname=file6)       #altitude of the surface (km)
HTOA   = 77.0       #altitude of the top of the atmosphere (km)

#--------------------------------------------------;
ISCAT_sw   = 0
ISTRM = 0   # 4 streams for sw
NSTR  = 4   # of streams

In [5]:
# =====RRTM input data (atmospheric and surface properties)=====
# ***Note: RRTM requires Altitude (ZM) to be in a ascending order!!! (Make sure the vertical order of all other variables is consistent with the order of ZM)
# ***Note: Be careful about the unit of each parameter (Check JCHAR variable in RRTM_SW instruction)

atm_prof = np.loadtxt('atm_profile_aerosol.txt',skiprows=1)
ZM = atm_prof[::-1,0] #km (boundary altitude (km))
PM = atm_prof[::-1,1] #hpa
TM = atm_prof[::-1,2] #K
h2o = atm_prof[::-1,4]  #g/m3   #kg/kg . kg/kg *1e3 = gm/kg
o3 = atm_prof[::-1,5]   #g/m3   #ppmv 'parts per million volume = 1e-6'. 1ppmv=1e-3*gm/kg
dext = atm_prof[::-1,6] #(km-1)


#Rs = None #np.array([0.1]) # surface albedo
#sza = None #np.array([22]) # solar zenith angle

In [6]:
# ===== Aerosol (e.g., dust) optical properties (i.e., Qext, SSA, g) =====
# ***Note: The order of 14 bands is listed in TABLE 1 in RRTM_SW instruction (It is not monotonically increase or decrease)
# ***Note: The order of spectral optical properties should be consistent with the order in TABLE 1

opt_dir = '../../Dust_Optical_Properties/'
wavelength_range = [0.35, 0.55, 0.75, 0.95] #[0.35, 0.55, 0.75, 0.95, 2, 4, 6, 8, 10, 12, 14, 16, 35]
opt_prop_sw = np.loadtxt(opt_dir + 'RRTM_bulk_dust_optical_properties_0W_18N.txt', skiprows=1)

qe_dust_sw = opt_prop_sw[:,0]  # Dust extinction efficienty (Qe) for 14 RRTM_SW bands
ssa_dust_sw = opt_prop_sw[:,1] # Dust single scattering albedo (SSA) for the 14 RRTM_SW bands
g_dust_sw = opt_prop_sw[:,2]   # Dust assymetric parameter (g) for the 14 RRTM_SW bands

In [7]:
print(qe_dust_sw)

[2.531 2.914 3.23  3.443 3.658 3.889 3.964 4.049 4.066 3.799 3.673 3.599
 3.413 1.011]


In [8]:
n_case = np.size(Rs)
print('*****Number of cases:',n_case)

for ic in range(n_case):
    SUF_ALB = np.ones(14)*Rs #np.ones(14)*Rs[ic]
    SZA = sza #sza[ic]
    if ZM[0]<0.1 :
        HBOUND = ZM[0]+0.001
    else:
        HBOUND = ZM[0]+0.01
    IMMAX = len(PM)
    IBMAX = IMMAX

    #****Note: The unit of each variable
    VMOL_wv = h2o   ###kg/kg*1e3=gm/kg
    VMOL_o3 = o3    ###ppmv*1e3=gm/kg
    VMOL_co2= np.ones(IMMAX)* 360 #no CO2 profile in data, add mannually, ppmv
    VMOL_n2o = np.ones(IMMAX)*0.3 #unit: PPMV
    VMOL_co = np.zeros(IMMAX)
    VMOL_ch4 = np.ones(IMMAX)*1.7 #unit: PPMV
    VMOL_total = np.vstack((VMOL_wv,VMOL_co2,VMOL_o3,VMOL_n2o,VMOL_co,VMOL_ch4))#density of the molecule set by JCHAR(K)
    VMOL=VMOL_total
    
    
    NLAY = np.count_nonzero(dext)   # Total number of layers containing the aerosol with the specified properties
    aer_lay = np.nonzero(dext)[0]+1 # Layer number  of aerosol layer. (layer n corresponds to the region between altitudes n and n+1 in the list of layer boundaries in Record 3.3B.)

    print(aer_lay)
    #----For each aerosol layer calculate AOD for each band----
    aod = np.zeros(NLAY)
    for i in range(NLAY):
        aod[i]=dext[aer_lay[i]-1]*(ZM[aer_lay[i]]-ZM[aer_lay[i]-1])
    aod_lay_band_sw=np.zeros((14,NLAY))
    for j in range(NLAY):
        aod_lay_band_sw[:,j]= aod[j] * qe_dust_sw/qe_dust_sw[9]   # qe_dust_sw[9] is qe @ 550nm. AOD(x) = AOD(550nm) * (qe(x)/qe(550nm)), where x is band number 
    
    print('*****Altitude for RRTM_SW (unit: km):',ZM)
    
    #========================== Run RRTM_SW ====================================
    ICLD = 0 # Cloud-free
    IAER = 0 # Aerosol-free
    print(' Compute clear-sky sw flux w/o aerososl')
    read_input_rrtm_sw('INPUT_RRTM', IAER, ICLD, ISCAT_sw, ISTRM, SZA, 1.0-SUF_ALB, \
    HBOUND, IMMAX, ZM, PM, TM, VMOL)
    os.system(RT_model_sw)
    os.system('mv INPUT_RRTM INPUT_RRTM.sw')
    os.system('mv OUTPUT_RRTM ./rrtm_output/OUTPUT_RRTM_sw_Rs{:}_sza{:}.clear'.format(Rs,sza))
    
    
    ICLD = 0   # Cloud-free 
    IAER = 10  # With Aerosol
    print(' Compute sw fluxs with aerosol')
    read_input_rrtm_sw('INPUT_RRTM', IAER, ICLD, ISCAT_sw, ISTRM, SZA, 1.0-SUF_ALB, \
    HBOUND, IMMAX, ZM, PM, TM, VMOL)
    write_IN_AER_RRTM_sw('IN_AER_RRTM',NLAY, aer_lay, aod_lay_band_sw,ssa_dust_sw,g_dust_sw)
    os.system(RT_model_sw)
    os.system('mv INPUT_RRTM INPUT_RRTM.sw')
    os.system('mv IN_AER_RRTM IN_AER_RRTM.sw')
    os.system('mv OUTPUT_RRTM ./rrtm_output/OUTPUT_RRTM_sw_Rs{:}_sza{:}.aerosol'.format(Rs,sza))
    

*****Number of cases: 1
[1 4 5 6]
*****Altitude for RRTM_SW (unit: km): [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 30. 35. 40. 45. 50.]
 Compute clear-sky sw flux w/o aerososl
 Compute sw fluxs with aerosol
